In [1]:
# import stuff here
import requests
import pymongo
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd


In [2]:
# Set up splinter
executable_path = {'executable_path':ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\Sharon\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
# NASA Mars News ....
# Use the splinter browser... this cell took waaayyy longer than it should have.
nasa_url = "https://redplanetscience.com/"

browser.visit(nasa_url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

results = soup.find_all('div', class_='list_text')

results

[<div class="list_text">
 <div class="list_date">April 10, 2021</div>
 <div class="content_title">NASA's Mars 2020 Will Hunt for Microscopic Fossils</div>
 <div class="article_teaser_body">A new paper identifies a ring of minerals at the rover's landing site that are ideal for fossilizing microbial life.</div>
 </div>,
 <div class="list_text">
 <div class="list_date">April 9, 2021</div>
 <div class="content_title">Mars 2020 Unwrapped and Ready for More Testing</div>
 <div class="article_teaser_body">In time-lapse video, bunny-suited engineers remove the inner layer of protective foil on NASA's Mars 2020 rover after it was relocated for testing.</div>
 </div>,
 <div class="list_text">
 <div class="list_date">April 6, 2021</div>
 <div class="content_title">Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover</div>
 <div class="article_teaser_body">NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's 

In [4]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.nasa_db
collection = db.items



In [5]:
# Loop through results and populate Mongo nasa_db
for result in results:
    try:
        title = result.find('div', class_='content_title').text
        
        blurb = result.find('div', class_='article_teaser_body').text
        
        print('**************')
        print(f'Title: {title}')
        print(f'Blurb: {blurb}')
        
        # gather up the items for mongo db
        if (title and blurb):
           post = {
               'title': title,
               'blurb': blurb,
            }

           collection.insert_one(post)
    except Exception as e:
        print(e)
    

**************
Title: NASA's Mars 2020 Will Hunt for Microscopic Fossils
Blurb: A new paper identifies a ring of minerals at the rover's landing site that are ideal for fossilizing microbial life.
**************
Title: Mars 2020 Unwrapped and Ready for More Testing
Blurb: In time-lapse video, bunny-suited engineers remove the inner layer of protective foil on NASA's Mars 2020 rover after it was relocated for testing.
**************
Title: Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover
Blurb: NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's entry for "Perseverance" was voted tops among 28,000 entries. 
**************
Title: NASA's Treasure Map for Water Ice on Mars
Blurb: A new study identifies frozen water just below the Martian surface, where astronauts could easily dig it up.
**************
Title: Air Deliveries Bring NASA's Perseverance Mars Rover Closer to Launch
Blurb: A NASA Wallops

In [6]:
# JPL Mars Space Images - Featured Image

jpl_url = "https://spaceimages-mars.com/"

browser.visit(jpl_url)

html = browser.html
jpl_soup = BeautifulSoup(html, 'html.parser')
jpl_soup


<html class=""><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
<!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<title>Space Image</title>
<style type="text/css">.fancybox-margin{margin-right:17px;}</style></head>
<body>
<div class="header">
<nav class="navbar navbar-expand-lg">
<a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"/><span class="logo">Jet Propulsion Laboratory</span>
<span class="logo1">California Institute of Technology</span></a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbarNav" data-toggle="collapse" type="button

In [7]:
#jpl_results = jpl_soup.find_all('div', class_='floating_text_area')
jpl_results = jpl_soup.find_all('img', class_='headerimage fade-in')

featured_image_url = f"{jpl_url}{jpl_results[0]['src']}"
featured_image_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

In [8]:
# Mars Facts
mars_facts_url = 'https://galaxyfacts-mars.com/'

tables = pd.read_html(mars_facts_url)
#tables
mars_earth_table_df = tables[0]
mars_earth_table_df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [9]:
mars_profile_table_df = tables[1]

In [10]:

#
me_list = mars_earth_table_df.loc[0]
mars_earth_table_df.columns = me_list

mars_earth_table_df

,Mars - Earth Comparison,Mars,Earth
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [11]:
print(mars_earth_table_df.to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars - Earth Comparison</th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Mars - Earth Comparison</td>
      <td>Mars</td>
      <td>Earth</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Diameter:</td>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Distance from Sun:</td>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Length of Year:</td>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Temperature:</td>
      <td>-87 to -5 °C</td>
      <td>-88 to 

In [12]:

mars_profile_table_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [13]:
mp_list =["Mars Attribute", "Measurement"]
mars_profile_table_df.columns = mp_list

print(mars_profile_table_df.to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars Attribute</th>
      <th>Measurement</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 ( Phobos &amp; Deimos )</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>

In [14]:
# Mars Hemispheres  - high resolutions images for each of Mars' hemispheres
hemi_base_url = "https://marshemispheres.com/images/"

# Valles Marineris
valles_marineris_url = f'{hemi_base_url}valles_marineris_enhanced.tif'

# Cerberus
cerberus_url = f'{hemi_base_url}cerberus_enhanced.tif'

# Schiaparelli 
schiaparelli_url = f'{hemi_base_url}schiaparelli_enhanced.tif'

# Syrtis Major
syrtis_major_url = f'{hemi_base_url}syrtis_major_enhanced.tif'


In [15]:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": valles_marineris_url},
    {"title": "Cerberus Hemisphere", "img_url": cerberus_url},
    {"title": "Schiaparelli Hemisphere", "img_url": schiaparelli_url},
    {"title": "Syrtis Major Hemisphere", "img_url": syrtis_major_url},
]

hemisphere_image_urls

[{'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced.tif'},
 {'title': 'Cerberus Hemisphere',
  'img_url': 'https://marshemispheres.com/images/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced.tif'}]

In [22]:
#cerberus_url = "https://marshemispheres.com/cerberus.html"
USGS_url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
base_USGS_url = "https://astrogeology.usgs.gov"
browser.visit(USGS_url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
#soup
results = soup.find_all('div', class_='item')
#results
hemi_img_urls = []

for result in results:
    title = result.find('h3').text
    href = result.find('a')['href']
    img_url = f'{base_USGS_url}{href}'
    print(img_url)
    browser.visit(img_url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    original_img = soup.find('div', class_='downloads')
    img_url = original_img.find('a')['href']
    hemi = {'title':title, 'img_url': img_url}
    hemi_img_urls.append(hemi)
    

https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced


In [23]:
hemi_img_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]